## Load the Data

Make sure the data file is on the path referenced below.
* You could mount your google drive and put the data file there.
* Or you could upload the data file to the Google Colab server
* Or maybe you are using Jupyter Notebook or an editor---either way, make sure the file path is valid

The first two lines are importing two libraries we will use almost every time:
* pandas: a package for using data tables, known as "DataFrames". This makes it easy to work with time-series data with dates, other labels, etc. It is one of the most used libraries in computational finance.
* numpy: a package for doing scientific computing in Python. Remember Python is a much more general language, and at its core does not need to be used for math. numpy is the collection of standard math mehtods.

By giving both pandas and numpy abbreviations (pd and np respectively) we make it more convenient to call the functions contained in their libraries.

In [1]:
import pandas as pd
import numpy as np

# Here, I assume that the data file is in the same directory as the the running
# Python interpeter. This often means the same directory as this code file.
# Change the path variable below to suit your particular setup.
path_to_data_file = '/content/drive/My Drive/Colab Notebooks/assetclass_data_monthly_2009.xlsx'
raw_data = pd.read_excel(path_to_data_file)

raw_data = raw_data.set_index('Dates')
columns_risky_assets = raw_data.columns[0:-1]
rets = raw_data.loc[:,columns_risky_assets]
rf = raw_data['Cash']
# get excess returns by subtracting rf from risky returns
# note that we can't subtract two tables as rets - rf, but need to use rets.subtract(rf,axis=0)
retsx = rets.subtract(rf, axis=0)
# view the last 3 rows of the table to see if as expected
retsx.tail()


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/My Drive/Colab Notebooks/assetclass_data_monthly_2009.xlsx'

### Summarize the data

In [ ]:
mu = retsx.mean()
# mu is the mean of each column, making it a row vector. To be consistent with 
# Pandas datatables are lazy about this, and will allow us to use mu as a column
# or a row, depending on what makes sense for matrix multiplication.

vol = retsx.std()
sharpe = mu / vol
summary = pd.DataFrame({'Mean':mu, 'Vol':vol, 'Sharpe': sharpe})
summary

,Mean,Vol,Sharpe
Domestic Equity,0.013028,0.037414,0.348220
Foreign Equity,0.008126,0.045516,0.178535
Emerging Markets,0.008033,0.058230,0.137954
Private Equity,0.013641,0.057616,0.236753
Absolute Return,0.001936,0.012769,0.151593
High Yield,0.007353,0.023927,0.307293
Commodities,-0.001676,0.055118,-0.030401
Real Estate,0.014564,0.050511,0.288341
Domestic Bonds,0.003093,0.016780,0.184348
Foreign Bonds,0.002109,0.022195,0.095042


## MV Tangency Portfolio

Here we take advantage of the formula for the tangency:

$\boldsymbol{w}^{\text{t}} = \Sigma^{-1}\tilde{\mu}\frac{1}{\boldsymbol{1}_n'\Sigma^{-1}\tilde{\mu}}$

But when we code this, we can ignore the scaling, and calculate

$\boldsymbol{w}^{\text{t}} \sim \Sigma^{-1}\tilde{\mu}$

Then simply adjust the vector to add up to 1, by dividing it by the sum of its unscaled elements.


In [ ]:
Sigma = retsx.cov()
# numpy as np at the top gives us access to computational math functions
# access them through np.

Sigma_inv = np.linalg.inv(Sigma)
# from the formula for the tangency weights
N = mu.shape[0]
weights = Sigma_inv @ mu / (np.ones(N) @ Sigma_inv @ mu)      


In [ ]:


# but the formula is just a complicated way to 
# make sure the vector adds to one. 
# Instead, calculate the vector, and divide it by its own sum

weights = Sigma_inv @ mu
weights = weights / weights.sum()

# For convenience, I'll wrap the solution back into a pandas.Series object.
# this prints it as a table with labels, or what pandas calls an index
# I want it to have the same labels as the summary stats above
wts_tan = pd.Series(weights, index=summary.index)

wts_tan

Domestic Equity      1.100132
Foreign Equity      -0.045800
Emerging Markets    -0.144565
Private Equity      -0.166304
Absolute Return     -1.166062
High Yield           0.791084
Commodities         -0.117513
Real Estate         -0.215180
Domestic Bonds       0.799114
Foreign Bonds       -0.022817
Inflation-Indexed    0.187910
dtype: float64

### Compute and display the weights of the tangency portfolio
Build a function to do this for any array of excess returns chosen.

## Compute the mean, volatility, and Sharpe ratio for the tangency.

Again, we could use the analytic formulas to get the statistics for the tangency portfolio, but I advise skipping this and calculating them directly as the next cell block does.


In [ ]:
mu_tan = mu @ wts_tan
vol = np.sqrt(wts_tan @ Sigma @ wts_tan)
sharpe_tan = np.sqrt(mu @ Sigma_inv @ mu.transpose())
sharpe_tan

0.6825681561145167

We can simply construct the month-by-month returns of the tangency portfolio, then take the statistics directly

In [ ]:
retsx_tan = retsx @ wts_tan
mu_tan = retsx_tan.mean()
vol_tan = retsx_tan.std()
sharpe_tan = mu_tan / vol_tan

Both will give the same result.

Add the tangency portfolio stats to our summary table by creating a new row, "Tangency".

In [ ]:
summary.loc['Tangency',:] = [mu_tan, vol_tan, sharpe_tan]
summary

,Mean,Vol,Sharpe
Domestic Equity,0.013028,0.037414,0.348220
Foreign Equity,0.008126,0.045516,0.178535
Emerging Markets,0.008033,0.058230,0.137954
Private Equity,0.013641,0.057616,0.236753
Absolute Return,0.001936,0.012769,0.151593
High Yield,0.007353,0.023927,0.307293
Commodities,-0.001676,0.055118,-0.030401
Real Estate,0.014564,0.050511,0.288341
Domestic Bonds,0.003093,0.016780,0.184348
Foreign Bonds,0.002109,0.022195,0.095042


As expected, the Tangency portfolio has a much higher Sharpe Ratio than the individual assets.

If an investor wants a higher or lower mean return, he/she can simply mix this tangency portfolio with the risk-free rate.

## Building a Function

What if we are doing some of these calculations routinely? 
Then we may prefer having a function do these calculations for us internally
and just return the answer to our workspace.

Let's test this out by building a function that takes

* input: dataframe of excess returns
* output: weight vector of tangency portfolio
* extra output: sharpe ratio of tangency portfolio

In [ ]:
def compute_tangency(retsx):
    """Compute tangency portfolio given a set of excess returns.

    Also, for convenience, this returns the associated vector of average
    returns and the variance-covariance matrix.
    
    """
    mu = retsx.mean()
    Sigma = retsx.cov()        
    Sigma_inv = np.linalg.inv(Sigma)
    
    weights = Sigma_inv @ mu
    weights = weights / weights.sum()
    
    wts_tan = pd.Series(weights, index=mu.index)
    
    retsx_tan = retsx @ wts_tan
    sharpe_tan = retsx_tan.mean()/retsx_tan.std()
    
    return wts_tan, sharpe_tan


With this function, we can now quickly calculate the tangency portfolio for any collection of excess returns.

The function returns two things: wts_tan and sharpe_tan. Thus, we can optionally put two variables on the left-hand-side, and it will return both.

You can see that the sharpe ratio of the tangency portfolio is the same as above, which verifies our function is calculating correctly.

In [ ]:
wts_tan = compute_tangency(retsx)
wts_tan, sharpe_tan = compute_tangency(retsx)
sharpe_tan

0.6825681561145187

The function enables us to redo all these calculations for subsets of the original assets.

In [ ]:
wts_tan_2016, sharpe_tan_2016 = compute_tangency(retsx.loc['2009':'2016',:])
print(wts_tan_2016)
print(f'\nSharpe ratio is: {sharpe_tan_2016}')

Domestic Equity      0.892357
Foreign Equity      -0.078261
Emerging Markets    -0.149629
Private Equity      -0.118068
Absolute Return     -0.647109
High Yield           0.604045
Commodities         -0.070434
Real Estate         -0.160209
Domestic Bonds       0.564514
Foreign Bonds       -0.118201
Inflation-Indexed    0.280995
dtype: float64

Sharpe ratio is: 0.7520647641821943


This calculation shows that using only return data up through 2016, the mean variance weights are different, and the sharpe ratio is higher.

This simply means that the return data in 2017-2019 is relatively weaker, so the full-sample (2009-2019) has a slightly lower Sharpe ratio.